In [4]:
#!/usr/bin/env python3
import gym
import rlkit.torch.pytorch_util as ptu
from rlkit.data_management.env_replay_buffer import EnvReplayBuffer
from rlkit.envs.wrappers import NormalizedBoxEnv
from rlkit.launchers.launcher_util import setup_logger
from rlkit.samplers.data_collector import MdpPathCollector
from rlkit.torch.sac.policies import TanhGaussianPolicy, MakeDeterministic
from rlkit.torch.sac.sac import SACTrainer
from rlkit.torch.networks import ConcatMlp
from rlkit.torch.torch_rl_algorithm import TorchBatchRLAlgorithm


def experiment(variant):
    print("Using GPU?: " + str(ptu.gpu_enabled()))
    expl_env = NormalizedBoxEnv(gym.make("MountainCarContinuous-v0"))
    eval_env = NormalizedBoxEnv(gym.make("MountainCarContinuous-v0"))
    obs_dim = expl_env.observation_space.low.size
    action_dim = eval_env.action_space.low.size

    M = variant['layer_size']
    qf1 = ConcatMlp(
        input_size=obs_dim + action_dim,
        output_size=1,
        # First layer nn.Linear(input_size,hidden_sizes[0])
        # 2nd layer nn.Linear(hidden_sizes[0], hidden_sizes[1])
        # Bias and initial Linear weights set with
        #  ptu.fanin_init and .1
        hidden_sizes=[M, M],
    )
    qf2 = ConcatMlp(
        input_size=obs_dim + action_dim,
        output_size=1,
        hidden_sizes=[M, M],
    )
    target_qf1 = ConcatMlp(
        input_size=obs_dim + action_dim,
        output_size=1,
        hidden_sizes=[M, M],
    )
    target_qf2 = ConcatMlp(
        input_size=obs_dim + action_dim,
        output_size=1,
        hidden_sizes=[M, M],
    )
    policy = TanhGaussianPolicy(
        obs_dim=obs_dim,
        action_dim=action_dim,
        hidden_sizes=[M, M],
    )
    eval_policy = MakeDeterministic(policy)
    eval_path_collector = MdpPathCollector(
        eval_env,
        eval_policy,
    )
    expl_path_collector = MdpPathCollector(
        expl_env,
        policy,
    )
    replay_buffer = EnvReplayBuffer(
        variant['replay_buffer_size'],
        expl_env,
    )
    trainer = SACTrainer(
        env=eval_env,
        policy=policy,
        qf1=qf1,
        qf2=qf2,
        target_qf1=target_qf1,
        target_qf2=target_qf2,
        **variant['trainer_kwargs']
    )
    algorithm = TorchBatchRLAlgorithm(
        trainer=trainer,
        exploration_env=expl_env,
        evaluation_env=eval_env,
        exploration_data_collector=expl_path_collector,
        evaluation_data_collector=eval_path_collector,
        replay_buffer=replay_buffer,
        **variant['algorithm_kwargs']
    )
    algorithm.to(ptu.device)
    algorithm.train()


    

In [ ]:
variant = dict(
        algorithm="SAC",
        version="normal",
        layer_size=256,
        replay_buffer_size=int(1E6),
        algorithm_kwargs=dict(
            num_epochs=10,#3000,
            num_eval_steps_per_epoch=5000,
            num_trains_per_train_loop=1000,
            num_expl_steps_per_train_loop=1000,
            min_num_steps_before_training=1000,
            max_path_length=1000,
            batch_size=256,
        ),
        trainer_kwargs=dict(
            discount=0.99,
            soft_target_tau=5e-3,
            target_update_period=1,
            policy_lr=3E-4,
            qf_lr=3E-4,
            reward_scale=1,
            use_automatic_entropy_tuning=True,
        ),
    )
setup_logger('example-sac-car', variant=variant,log_dir="./rlkit_out/")
ptu.set_gpu_mode(True)  # optionally set the GPU (default=False)
experiment(variant)

2021-03-01 09:20:18.550910 UTC | Variant:
2021-03-01 09:20:18.551866 UTC | {
  "algorithm": "SAC",
  "version": "normal",
  "layer_size": 256,
  "replay_buffer_size": 1000000,
  "algorithm_kwargs": {
    "num_epochs": 3000,
    "num_eval_steps_per_epoch": 5000,
    "num_trains_per_train_loop": 1000,
    "num_expl_steps_per_train_loop": 1000,
    "min_num_steps_before_training": 1000,
    "max_path_length": 1000,
    "batch_size": 256
  },
  "trainer_kwargs": {
    "discount": 0.99,
    "soft_target_tau": 0.005,
    "target_update_period": 1,
    "policy_lr": 0.0003,
    "qf_lr": 0.0003,
    "reward_scale": 1,
    "use_automatic_entropy_tuning": true
  }
}
Using GPU?: True
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box a

/opt/conda/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


2021-03-01 09:21:02.945961 UTC | [] Epoch 0 finished
----------------------------------  --------------
replay_buffer/size                  2000
trainer/num train calls             1000
trainer/QF1 Loss                       0.432287
trainer/QF2 Loss                       0.432451
trainer/Policy Loss                   -0.677446
trainer/Q1 Predictions Mean            0.000107123
trainer/Q1 Predictions Std             6.85158e-05
trainer/Q1 Predictions Max             0.000292433
trainer/Q1 Predictions Min             3.27804e-06
trainer/Q2 Predictions Mean           -2.56457e-05
trainer/Q2 Predictions Std             4.39631e-05
trainer/Q2 Predictions Max             7.51978e-05
trainer/Q2 Predictions Min            -0.000127115
trainer/Q Targets Mean                 0.61744
trainer/Q Targets Std                  0.226243
trainer/Q Targets Max                  2.08988
trainer/Q Targets Min                  0.172897
trainer/Log Pis Mean                  -0.67747
trainer/Log Pis Std      